In [1]:
import os
import time
import math
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from masked_cross_entropy import masked_cross_entropy

In [2]:
USE_CUDA = torch.cuda.is_available()
USE_CUDA

False

In [3]:
base_dir = 'data/twitter'
train_dir = os.path.join(base_dir, 'twitter_clean.txt')
vocab_dir = os.path.join(base_dir, 'twitter_vocab.txt')

In [4]:
class Corpus(object):
    """
    文本预处理，获取词汇表，并将字符串文本转换为数字序列。
    """

    def __init__(self, train_dir, vocab_dir):
        assert os.path.exists(train_dir), 'File %s does not exist.' % train_dir
        assert os.path.exists(vocab_dir), 'File %s does not exist.' % vocab_dir

        words = open(vocab_dir, encoding='utf-8').read().strip().split('\n')
        word_to_id = dict(zip(words, range(len(words))))
        
        assert word_to_id['<pad>'] == 0, "<pad> id should be 0."
        
        self.words = words
        self.word_to_id = word_to_id
        
        self.tokenize(train_dir)
        
    def tokenize(self, train_dir):
        data = open(train_dir, encoding='utf-8').read().strip().split('\n')
        questions, answers = [], []
        for line in data:
            question, answer = line.split(" ==> ")
            questions.append(self.text_to_ids(question))
            answers.append(self.text_to_ids(answer))
            
        total_num = len(questions)
        train_num = int(0.9 * total_num)
        self.x_train, self.y_train = questions[:train_num], answers[:train_num]
        self.x_test, self.y_test = questions[train_num:], answers[train_num:]
        
    def text_to_ids(self, text):
        return [self.word_to_id[x] for x in (text.split() + ['<eos>'])]
    
    def ids_to_text(self, ids):
        return [self.words[x] for x in ids]

    def __repr__(self):
        return "Train length: %d, Test length: %d, Vocabulary size: %d" % (len(self.x_train), 
                                                                           len(self.x_test), 
                                                                           len(self.words))

In [5]:
class DataSet(object):
    def __init__(self, data, labels, batch_size=64):
        num_batches = len(data) // batch_size
        data, labels = data[:(num_batches * batch_size)], labels[:(num_batches * batch_size)]
        self.data = []
        for i in range(num_batches):
            x_batch = data[(i * batch_size):((i+1)*batch_size)]
            y_batch = labels[(i * batch_size):((i+1)*batch_size)]
            x_pad, x_lengths, y_pad, y_lengths = self.pad_batch(x_batch, y_batch)
            self.data.append((x_pad, x_lengths, y_pad, y_lengths))
            
    def pad_batch(self, x_batch, y_batch):
        seq_pairs = sorted(zip(x_batch, y_batch), key=lambda p: len(p[0]), reverse=True)
        x_batch, y_batch = zip(*seq_pairs)
        
        x_lengths = list(map(len, x_batch))
        x_padded = [self.pad_seq(s, max(x_lengths)) for s in x_batch]
        
        y_lengths = list(map(len, y_batch))
        y_padded = [self.pad_seq(s, max(y_lengths)) for s in y_batch]
        
        input_var = Variable(torch.LongTensor(x_padded)).transpose(0, 1)
        target_var = Variable(torch.LongTensor(y_padded)).transpose(0, 1)
        
        if USE_CUDA:
            input_var = input_var.cuda()
            target_var = target_var.cuda()
            
        return input_var, x_lengths, target_var, y_lengths
    
    def pad_seq(self, seq, max_len):
        return seq + [0] * (max_len - len(seq))
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

In [6]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout)
        
    def forward(self, input_seqs, input_lengths, hidden=None):
        embedded = self.embedding(input_seqs)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        return outputs, hidden

In [7]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(0)
        this_batch_size = encoder_outputs.size(1)

        # Create variable to store attention energies
        attn_energies = Variable(torch.zeros(this_batch_size, max_len)) # B x S

        if USE_CUDA:
            attn_energies = attn_energies.cuda()

        # For each batch of encoder outputs
        for b in range(this_batch_size):
            # Calculate energy for each encoder output
            for i in range(max_len):
                attn_energies[b, i] = self.score(hidden[:, b], encoder_outputs[i, b].unsqueeze(0))

        # Normalize energies to weights in range 0 to 1, resize to 1 x B x S
        return F.softmax(attn_energies, dim=1).unsqueeze(1)
    
    def score(self, hidden, encoder_output):
        
        if self.method == 'dot':
            energy = hidden.dot(encoder_output)
            return energy
        
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = hidden.dot(energy)
            return energy
        
        elif self.method == 'concat':
            energy = self.attn(torch.cat((hidden, encoder_output), 1))
            energy = self.v.dot(energy)
            return energy

In [8]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, hidden_size, output_size, embedding, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout)
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
        # Choose attention model
        if attn_model != 'none':
            self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_seq, last_hidden, encoder_outputs):
        # Note: we run this one step at a time

        # Get the embedding of the current input word (last output word)
        batch_size = input_seq.size(0)
        embedded = self.embedding(input_seq)
        embedded = self.embedding_dropout(embedded)
        embedded = embedded.view(1, batch_size, self.hidden_size) # S=1 x B x N

        # Get current hidden state from input word and last hidden state
        rnn_output, hidden = self.gru(embedded, last_hidden)

        # Calculate attention from current RNN state and all encoder outputs;
        # apply to encoder outputs to get weighted average
        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x S=1 x N

        # Attentional vector using the RNN hidden state and context vector
        # concatenated together (Luong eq. 5)
        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
        context = context.squeeze(1)       # B x S=1 x N -> B x N
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = F.tanh(self.concat(concat_input))

        # Finally predict next token (Luong eq. 6, without softmax)
        output = self.out(concat_output)

        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights

In [9]:
def train(input_batches, input_lengths, target_batches, target_lengths, encoder, decoder, encoder_optimizer, decoder_optimizer):
    
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 # Added onto for each word

    # Run words through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)
    
    # Prepare input and output variables
    decoder_input = Variable(torch.LongTensor([corpus.word_to_id['<sos>']] * batch_size))
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder

    max_target_length = max(target_lengths)
    all_decoder_outputs = Variable(torch.zeros(max_target_length, batch_size, decoder.output_size))

    # Move new Variables to CUDA
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        all_decoder_outputs = all_decoder_outputs.cuda()

    # Run through decoder one time step at a time
    for t in range(max_target_length):
        decoder_output, decoder_hidden, decoder_attn = decoder(
            decoder_input, decoder_hidden, encoder_outputs)

        all_decoder_outputs[t] = decoder_output
        decoder_input = target_batches[t] # Next input is current target

    # Loss calculation and backpropagation
    loss = masked_cross_entropy(
        all_decoder_outputs.transpose(0, 1).contiguous(), # -> batch x seq
        target_batches.transpose(0, 1).contiguous(), # -> batch x seq
        target_lengths,
        USE_CUDA
    )
    loss.backward()
    
    # Clip gradient norms
    ec = torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    dc = torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)

    # Update parameters with optimizers
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0], ec, dc

In [10]:
def evaluate(input_seq, max_length=20):
    input_lengths = [len(input_seq)]
    input_batches = Variable(torch.LongTensor([input_seq]), volatile=True).transpose(0, 1)
    
    if USE_CUDA:
        input_batches = input_batches.cuda()
        
    # Set to not-training mode to disable dropout
    encoder.eval()
    decoder.eval()
    
    # Run through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)

    # Create starting vectors for decoder
    decoder_input = Variable(torch.LongTensor([corpus.word_to_id['<sos>']]), volatile=True) # SOS
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder
    
    if USE_CUDA:
        decoder_input = decoder_input.cuda()

    # Store output words and attention states
    decoded_words = []
    decoder_attentions = torch.zeros(max_length + 1, max_length + 1)
    
    # Run through decoder
    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )
        decoder_attentions[di,:decoder_attention.size(2)] += decoder_attention.squeeze(0).squeeze(0).cpu().data
        
        # Choose top word from output
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == corpus.word_to_id['<eos>']:
            decoded_words.append('<eos>')
            break
        else:
            decoded_words.append(corpus.words[ni])
            
        # Next input is chosen word
        decoder_input = Variable(torch.LongTensor([ni]))
        if USE_CUDA: decoder_input = decoder_input.cuda()

    # Set back to training mode
    encoder.train()
    decoder.train()
    
    return decoded_words

In [11]:
def evaluate_randomly():
    index = random.choice(range(len(corpus.x_test)))
    input_seq, target_seq = corpus.x_test[index], corpus.y_test[index]
    output_words = evaluate(input_seq)
    print("Input:", ' '.join(corpus.ids_to_text(input_seq)))
    print("Target:", ' '.join(corpus.ids_to_text(target_seq)))
    print("Predicted:", ' '.join(output_words))

In [12]:
corpus = Corpus(train_dir, vocab_dir)
corpus

Train length: 194785, Test length: 21643, Vocabulary size: 10000

In [13]:
# Configure models
attn_method = 'dot'
hidden_size = 500
n_layers = 2
dropout = 0.1
batch_size = 100

train_data = DataSet(corpus.x_train, corpus.y_train, batch_size)

In [14]:
# Configure training/optimization
clip = 50.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_epochs = 50000
epoch = 0
plot_every = 20
print_every = 3
evaluate_every = 6

vocab_size = len(corpus.words)
embedding = nn.Embedding(vocab_size, hidden_size)

# Initialize models
encoder = EncoderRNN(hidden_size, embedding, n_layers, dropout=dropout)
decoder = LuongAttnDecoderRNN(attn_method, hidden_size, vocab_size, embedding, n_layers, dropout=dropout)

# Initialize optimizers and criterion
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

# Move models to GPU
if USE_CUDA:
    encoder.cuda()
    decoder.cuda()

# Keep track of time elapsed and running averages
start = time.time()
plot_losses = []
print_loss_total = 0 # Reset every print_every
plot_loss_total = 0 # Reset every plot_every

In [15]:
def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

In [ ]:
# Begin!
ecs = []
dcs = []
eca = 0
dca = 0

while epoch < n_epochs:
    epoch += 1
    
    # Get training data for this cycle
    input_batches, input_lengths, target_batches, target_lengths = random.choice(train_data)

    # Run the train function
    loss, ec, dc = train(
        input_batches, input_lengths, target_batches, target_lengths,
        encoder, decoder,
        encoder_optimizer, decoder_optimizer
    )

    # Keep track of loss
    print_loss_total += loss
    plot_loss_total += loss
    eca += ec
    dca += dc
    
    # job.record(epoch, loss)

    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg)
        print(print_summary)
        
    if epoch % evaluate_every == 0:
        evaluate_randomly()
        evaluate_randomly()

#     if epoch % plot_every == 0:
#         plot_loss_avg = plot_loss_total / plot_every
#         plot_losses.append(plot_loss_avg)
#         plot_loss_total = 0
        
#         # TODO: Running average helper
#         ecs.append(eca / plot_every)
#         dcs.append(dca / plot_every)
#         ecs_win = 'encoder grad (%s)' % hostname
#         dcs_win = 'decoder grad (%s)' % hostname
#         vis.line(np.array(ecs), win=ecs_win, opts={'title': ecs_win})
#         vis.line(np.array(dcs), win=dcs_win, opts={'title': dcs_win})
#         eca = 0
#         dca = 0

In [ ]:
evaluate_randomly()

In [ ]:
torch.save(encoder.state_dict(), 'encoder_attn.pt')
torch.save(decoder.state_dict(), 'decoder_attn.pt')
torch.save(embedding.state_dict(), 'embedding_attn.pt')

In [ ]:
vocab_size = len(corpus.words)
embedding2 = nn.Embedding(vocab_size, hidden_size)

# Initialize models
encoder2 = EncoderRNN(hidden_size, embedding2, n_layers, dropout=dropout)
decoder2 = LuongAttnDecoderRNN(attn_method, hidden_size, vocab_size, embedding2, n_layers, dropout=dropout)

In [ ]:
embedding2.load_state_dict(torch.load('embedding_attn.pt', map_location=lambda storage, loc: storage))
encoder2.load_state_dict(torch.load('encoder_attn.pt', map_location=lambda storage, loc: storage))
decoder2.load_state_dict(torch.load('decoder_attn.pt', map_location=lambda storage, loc: storage))

if USE_CUDA:
    embedding2.cuda()
    encoder2.cuda()
    decoder2.cuda()

In [ ]:
def evaluate2(encoder2, decoder2, input_seq, max_length=20):
    input_lengths = [len(input_seq)]
    input_batches = Variable(torch.LongTensor([input_seq]), volatile=True).transpose(0, 1)
    
    if USE_CUDA:
        input_batches = input_batches.cuda()
        
    # Set to not-training mode to disable dropout
    encoder2.eval()
    decoder2.eval()
    
    # Run through encoder
    encoder_outputs, encoder_hidden = encoder2(input_batches, input_lengths, None)

    # Create starting vectors for decoder
    decoder_input = Variable(torch.LongTensor([corpus.word_to_id['<sos>']]), volatile=True) # SOS
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder
    
    if USE_CUDA:
        decoder_input = decoder_input.cuda()

    # Store output words and attention states
    decoded_words = []
    
    # Run through decoder
    for di in range(max_length):
        decoder_output, decoder_hidden = decoder2(
            decoder_input, decoder_hidden
        )

        # Choose top word from output
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == corpus.word_to_id['<eos>']:
            decoded_words.append('<eos>')
            break
        else:
            decoded_words.append(corpus.words[ni])
            
        # Next input is chosen word
        decoder_input = Variable(torch.LongTensor([ni]))
        if USE_CUDA: decoder_input = decoder_input.cuda()

    # Set back to training mode
    encoder2.train()
    decoder2.train()
    
    return decoded_words

In [ ]:
def evaluate_randomly2():
    index = random.choice(range(len(corpus.x_test)))
    input_seq, target_seq = corpus.x_test[index], corpus.y_test[index]
    output_words = evaluate2(encoder2, decoder2, input_seq)
    print("Input:", ' '.join(corpus.ids_to_text(input_seq)))
    print("Target:", ' '.join(corpus.ids_to_text(target_seq)))
    print("Predicted:", ' '.join(output_words))

In [ ]:
evaluate_randomly2()

In [ ]:
input